# H2O AutoML Regression Demo

This is a [Jupyter](https://jupyter.org/) Notebook. When you execute code within the notebook, the results appear beneath the code. To execute a code chunk, place your cursor on the cell and press *Shift+Enter*. 

### Start H2O

Import the **h2o** Python module and `H2OAutoML` class and initialize a local H2O cluster.

In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 mins 15 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.9
H2O cluster version age:,6 hours and 50 minutes
H2O cluster name:,H2O_from_python_navdeepgill_wzl1ou
H2O cluster total nodes:,1
H2O cluster free memory:,3.417 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### Load Data

For the AutoML regression demo, we use the [Combined Cycle Power Plant](http://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant) dataset.  The goal here is to predict the energy output (in megawatts), given the temperature, ambient pressure, relative humidity and exhaust vacuum values.  In this demo, you will use H2O's AutoML to outperform the [state of the art results](https://www.sciencedirect.com/science/article/pii/S0142061514000908) on this task.

In [2]:
# Use local data file or download from GitHub
import os
docker_data_path = "/home/h2o/data/automl/powerplant_output.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "../data/powerplant_output.csv"


# Load data into H2O
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Let's take a look at the data.

In [3]:
df.describe()

Rows:9568
Cols:5




,TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
type,real,real,real,real,real
mins,1.81,25.36,992.89,25.56,420.26
mean,19.651231187290968,54.30580372073578,1013.2590781772575,73.30897784280936,454.36500940635455
maxs,37.11,81.56,1033.3,100.16,495.76
sigma,7.452473229611079,12.707892998326809,5.938783705811604,14.600268756728953,17.066994999803416
zeros,0,0,0,0,0
missing,0,0,0,0,0
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.4,1012.16,92.14,488.56


Next, let's identify the response column and save the column name as `y`.  In this dataset, we will use all columns except the response as predictors, so we can skip setting the `x` argument explicitly.

In [4]:
y = "HourlyEnergyOutputMW"

Lastly, let's split the data into two frames, a `train` (80%) and a `test` frame (20%).  The `test` frame will be used to score the leaderboard and to demonstrate how to generate predictions using an AutoML leader model.

In [5]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

## Run AutoML 

Run AutoML, stopping after 60 seconds.  The `max_runtime_secs` argument provides a way to limit the AutoML run by time.  When using a time-limited stopping criterion, the number of models train will vary between runs.  If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another. 

The `test` frame is passed explicitly to the `leaderboard_frame` argument here, which means that instead of using cross-validated metrics, we use test set metrics for generating the leaderboard.

In [6]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_lb_frame")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


*Note: If you see the following error, it means that you need to install the pandas module.*
```
H2OTypeError: Argument `python_obj` should be a None | list | tuple | dict | numpy.ndarray | pandas.DataFrame | scipy.sparse.issparse, got H2OTwoDimTable 
``` 

For demonstration purposes, we will also execute a second AutoML run, this time providing the original, full dataset, `df` (without passing a `leaderboard_frame`).  This is a more efficient use of our data since we can use 100% of the data for training, rather than 80% like we did above.  This time our leaderboard will use cross-validated metrics.

*Note: Using an explicit `leaderboard_frame` for scoring may be useful in some cases, which is why the option is available.*  

In [7]:
aml2 = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_full_data")
aml2.train(y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


*Note: We specify a `project_name` here for clarity.*

## Leaderboard

Next, we will view the AutoML Leaderboard.  Since we specified a `leaderboard_frame` in the `H2OAutoML.train()` method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.

After viewing the `"powerplant_lb_frame"` AutoML project leaderboard, we compare that to the leaderboard for the `"powerplant_full_data"` project.  We can see that the results are better when the full dataset is used for training.  

A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric.  In the case of regression, the default ranking metric is mean residual deviance.  In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

In [8]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mae,rmsle
StackedEnsemble_AllModels_0_AutoML_20180511_164445,11.5788,3.40276,2.37623,0.00748815
GBM_grid_0_AutoML_20180511_164445_model_3,11.6056,3.4067,2.39691,0.00749595
StackedEnsemble_BestOfFamily_0_AutoML_20180511_164445,11.822,3.43832,2.40625,0.00757066
GBM_grid_0_AutoML_20180511_164445_model_2,12.1994,3.49276,2.4912,0.00767829
GBM_grid_0_AutoML_20180511_164445_model_1,12.4343,3.52623,2.53157,0.00775088
GBM_grid_0_AutoML_20180511_164445_model_0,12.7143,3.56571,2.56442,0.00783178
GBM_grid_0_AutoML_20180511_164445_model_12,13.3382,3.65215,2.65886,0.00802911
GBM_grid_0_AutoML_20180511_164445_model_10,13.432,3.66497,2.62157,0.00805937
DRF_0_AutoML_20180511_164445,13.5359,3.67911,2.60188,0.00810274
XRT_0_AutoML_20180511_164445,13.7162,3.70354,2.61325,0.0081558


Now we will view a snapshot of the top models.  Here we should see the two Stacked Ensembles at or near the top of the leaderboard.  Stacked Ensembles can almost always outperform a single model.

In [9]:
aml2.leaderboard.head()

model_id,mean_residual_deviance,rmse,mae,rmsle
StackedEnsemble_AllModels_0_AutoML_20180511_164553,10.6085,3.25706,2.33858,0.00715332
StackedEnsemble_BestOfFamily_0_AutoML_20180511_164553,11.0249,3.32038,2.38028,0.00729309
GBM_grid_0_AutoML_20180511_164553_model_3,11.0759,3.32805,2.39632,0.00730626
GBM_grid_0_AutoML_20180511_164553_model_2,11.4378,3.38198,2.45511,0.00742139
GBM_grid_0_AutoML_20180511_164553_model_1,11.6473,3.41281,2.48028,0.00748708
GBM_grid_0_AutoML_20180511_164553_model_0,11.9098,3.45105,2.52075,0.00756582
DRF_0_AutoML_20180511_164553,12.3347,3.51208,2.54641,0.00771637
GBM_grid_0_AutoML_20180511_164553_model_10,12.4785,3.53249,2.58537,0.00775737
XRT_0_AutoML_20180511_164553,12.5758,3.54624,2.5782,0.00778691
GBM_grid_0_AutoML_20180511_164553_model_4,13.2203,3.63597,2.68794,0.00797724


This dataset comes from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant) of machine learning datasets.  The data was used in a [publication](https://www.sciencedirect.com/science/article/pii/S0142061514000908) in the *International Journal of Electrical Power & Energy Systems* in 2014.  In the paper, the authors achieved a mean absolute error (MAE) of 2.818 and a Root Mean-Squared Error (RMSE) of 3.787 on their best model.  So, with H2O's AutoML, we've already beaten the state-of-the-art in just 60 seconds of compute time!

## Predict Using Leader Model

If you need to generate predictions on a test set, you can make predictions on the `"H2OAutoML"` object directly, or on the leader model object.

In [10]:
pred = aml.predict(test)
pred.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
488.533
475.095
464.258
453.39
447.563
470.222
443.293
463.776
443.216
432.308


If needed, the standard `model_performance()` method can be applied to the AutoML leader model and a test set to generate an H2O model performance object.

In [11]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 11.57878310773789
RMSE: 3.402761100597262
MAE: 2.376234301400711
RMSLE: 0.007488154430356533
R^2: 0.960386767436272
Mean Residual Deviance: 11.57878310773789
Null degrees of freedom: 1890
Residual degrees of freedom: 1879
Null deviance: 552841.2070770501
Residual deviance: 21895.47885673235
AIC: 10023.81428692889
